# Policy Consistency Validation Notebook

This notebook validates generated IDS policies against the expected IDS ontology. It is organized as follows:

- **Setup & Imports**: Import libraries and set paths.
- **Data Loading**: Load the generated policies from an Excel file.
- **Validation Functions**: Define functions to validate the policy JSON structure.
- **Policy Validation**: Apply validation to each generated policy.
- **Validation Summary**: Summarize and visualize the validation results.
- **Conclusion**: Final remarks and next steps.

In [ ]:
import os
import json
import re
import time
import pandas as pd
import matplotlib.pyplot as plt

# Set the path for the generated policies file (update as necessary)
data_file = 'Generated_Policies.xlsx'
print("Setup complete. Libraries imported.")

In [ ]:
# Load the generated policies from the Excel file
try:
    df = pd.read_excel(data_file)
    print(f"Data loaded successfully. Shape: {df.shape}")
except Exception as e:
    print(f"Error loading data: {e}")

# Display the first few rows for verification
print(df.head(10))

In [ ]:
def validate_policy(policy):
    """
    Validate an IDS policy for consistency with the IDS ontology.

    Parameters:
      policy (dict): The policy JSON object.

    Returns:
      tuple: (bool, str) where the boolean indicates validity and the string provides a message.
    """
    if not isinstance(policy, dict):
        return False, "Policy is not a dictionary."
    if "policy" not in policy:
        return False, "Missing top-level 'policy' key."

    pol = policy["policy"]
    if "permissions" not in pol:
        return False, "Missing 'permissions' key in policy."
    if not isinstance(pol["permissions"], list) or not pol["permissions"]:
        return False, "'permissions' should be a non-empty list."

    for perm in pol["permissions"]:
        if not isinstance(perm, dict):
            return False, "Each permission must be a dictionary."
        required_keys = ["assignee", "action", "constraints"]
        for key in required_keys:
            if key not in perm:
                return False, f"Permission missing required key: '{key}'."
        if not isinstance(perm["constraints"], list) or not perm["constraints"]:
            return False, "Each permission should have a non-empty 'constraints' list."
        for constraint in perm["constraints"]:
            if not isinstance(constraint, dict):
                return False, "Each constraint must be a dictionary."
            for c_key in ["leftOperand", "operator", "rightOperand"]:
                if c_key not in constraint:
                    return False, f"Constraint missing key: '{c_key}'."
    return True, "Policy is valid."

def validate_policy_json(policy_str):
    """
    Parse a policy JSON string and validate its structure.

    Parameters:
      policy_str (str): The generated policy as a JSON-formatted string.

    Returns:
      tuple: (bool, str) indicating whether the policy is valid and an associated message.
    """
    try:
        policy = json.loads(policy_str)
    except Exception as e:
        return False, f"Error parsing JSON: {e}"

    return validate_policy(policy)

print("Validation functions defined.")

In [ ]:
# Ensure that the 'Generated_Policy' column exists in the DataFrame
if 'Generated_Policy' not in df.columns:
    print("Error: 'Generated_Policy' column not found in the data.")
else:
    # Apply the validation function to each policy in the DataFrame
    validation_results = df['Generated_Policy'].apply(
        lambda x: validate_policy_json(x) if pd.notnull(x) else (False, "No policy provided")
    )

    # Create new columns for validation status and message
    df['Validation_Status'] = validation_results.apply(lambda tup: tup[0])
    df['Validation_Message'] = validation_results.apply(lambda tup: tup[1])

    # Display a sample of the validation results
    print(df[['Policy', 'Generated_Policy', 'Validation_Status', 'Validation_Message']].head(10))

In [ ]:
# Count valid vs. invalid policies
validation_summary = df['Validation_Status'].value_counts()
print("Validation Summary:")
print(validation_summary)

# Visualize the results as a bar chart
plt.figure(figsize=(6, 4))
validation_summary.plot(kind='bar')
plt.title("Validation Status of Generated Policies")
plt.xlabel("Valid (True) / Invalid (False)")
plt.ylabel("Number of Policies")
plt.show()

## Conclusion

This notebook validated the generated IDS policies against the expected ontology structure. The summary and visualizations indicate which policies adhere to the IDS standards and which require further review. Future work could involve refining policy generation or implementing more granular validation rules.